# **Double Entries For Internal Transfers**

# **Step1 - Identify Journal Entry Logic**

## The result for the double entry will be in the following format:

| je | pos |    date  | acct |     acct_descr | amt | dr | cr |
|:--:|:---:|:--------:|:----:|:--------------:|:---:|:--:|:--:|
|1   |  1  |2023-01-01| 10000|       Control  | 100 | 100| 00 |
|1   |  2  |2023-01-01| 3026 |Shareholder loan|-100 |  00|-100|
|1   |  3  |    00    |  00  |         -      |  00 |  00| 00 |
|1   |  4  |2023-01-02| 1000 |Cash & deposit  | 100 | 100| 00 |
|1   |  5  |2023-01-02| 10000|       Control  |-100 |  00|-100|
|1   |  6  |    00    |  00  |         -      |  00 |  00| 00 |
|----|-----|----------|------|----------------|-----|----|----|
|2   |  1  |2023-01-03| 10000|       Control  | 200 | 200| 00 |
|2   |  2  |2023-01-03| 1000 |Cash & deposit  |-200 |  00|-200|
|2   |  3  |    00    |  00  |         -      |  00 |  00| 00 |
|2   |  4  |2023-01-04| 1000 |Cash & deposit  | 200 | 200| 00 |
|2   |  5  |2023-01-04| 10000|       Control  |-200 |  00|-200|
|2   |  6  |    00    |  00  |         -      |  00 |  00| 00 |
|----|-----|----------|------|----------------|-----|----|----|
|3   |  1  |2023-01-05| 10000|       Control  | 300 | 300| 00 |
|3   |  2  |2023-01-05| 1000 |Shareholder loan|-300 |  00|-300|
|3   |  3  |    00    |  00  |         -      |  00 |  00| 00 |
|3   |  4  |2023-01-06| 1000 |Cash & deposit  | 300 | 300| 00 |
|3   |  5  |2023-01-06| 10000|       Control  |-300 |  00|-300|
|3   |  6  |    00    |  00  |         -      |  00 |  00| 00 |
|----|-----|----------|------|----------------|-----|----|----|

# **Step2 - Identify Data Source and required Python packages**

In [1]:
# Step 2.1 --- identify source data location
src_file = r"g:/example_interbank_raw_data.xlsx"

# Step 2.2 --- identify Python packages required 
import pandas as pd
import numpy as np
pd.set_option('display.max_columns',15)

# Step 2.3 --- create dataframe, 'df', to store the table from source file
df = pd.read_excel(src_file, dtype_backend='pyarrow').loc[:, 'chqdate':'deposit_amt'].dropna(how='all')
df

,chqdate,withdraw_bank,wbank_gl,withdraw_item,withdrawn_amt,deposit_date,deposit_bank,dbank_gl,deposit_item,deposit_amt
0,2023-01-01 00:00:00,BMO,3026,Shareholder loan,100,2023-01-02 00:00:00,HSBC,1000,Cash & deposit,100
1,2023-01-03 00:00:00,Royal Bank,1000,Cash & deposit,200,2023-01-04 00:00:00,HSBC,1000,Cash & deposit,200
2,2023-01-05 00:00:00,BNS,3026,Shareholder loan,300,2023-01-06 00:00:00,HSBC,1000,Cash & deposit,300


# **Step3 - Create'df1' (working table), 'jetb' (result table), and the key column; linked up with column,'je'**

In [2]:
# step 3.1 : assign journal entry numbers,'je', to each inter-bank transaction
df1 = df.assign(je = df.index + 1)
df1

,chqdate,withdraw_bank,wbank_gl,withdraw_item,withdrawn_amt,deposit_date,deposit_bank,dbank_gl,deposit_item,deposit_amt,je
0,2023-01-01 00:00:00,BMO,3026,Shareholder loan,100,2023-01-02 00:00:00,HSBC,1000,Cash & deposit,100,1
1,2023-01-03 00:00:00,Royal Bank,1000,Cash & deposit,200,2023-01-04 00:00:00,HSBC,1000,Cash & deposit,200,2
2,2023-01-05 00:00:00,BNS,3026,Shareholder loan,300,2023-01-06 00:00:00,HSBC,1000,Cash & deposit,300,3


In [3]:
# step 3.2 --- create a table, 'jetb' to hold the final result of journal entries
jetb = pd.DataFrame(columns=['je','pos','date','acct','item','amt','dr','cr'])
jetb

,je,pos,date,acct,item,amt,dr,cr
